In [16]:
#import libraries
import numpy as np
import pandas as pd 

In [17]:
#import database: 2 csv files 
prices = pd.read_csv("prices.csv")
sales = pd.read_csv("sales.csv")

In [20]:
#change data type to datetime & sorting based on updated date & ordered date
prices["updated_at"] = pd.to_datetime(prices["updated_at"])
sales["ordered_at"] = pd.to_datetime(sales["ordered_at"])
prices_sorted = prices.sort_values(by =["updated_at"])
sales_sorted = sales.sort_values(by =["ordered_at"])
print(sales_sorted)
print(prices_sorted)

     product_id          ordered_at  quantity_ordered
86      3954203 2018-09-11 01:43:00                 1
28      4085861 2018-09-11 06:26:00                 1
26      4085861 2018-09-11 06:53:00                 1
27      4085861 2018-09-11 08:24:00                 1
123     4085861 2018-09-11 09:30:00                 1
..          ...                 ...               ...
67      4085861 2018-09-18 20:23:00                 1
77      4085861 2018-09-18 20:43:00                 1
79      4085861 2018-09-18 20:54:00                 1
87      3954203 2018-09-18 21:26:00                 1
11      3998909 2018-09-18 22:11:00                 1

[175 rows x 3 columns]
    product_id  old_price  new_price          updated_at
4      3954203      68800      60000 2018-09-10 16:32:00
7      3998909      19000      17000 2018-09-10 16:35:00
0           64     270000     239000 2018-09-10 16:37:00
11     4085861      60000      53500 2018-09-11 08:51:00
1      3954203      60000      64000 2018-0

In [23]:
#determine the nearest update price for each order
data = pd.merge_asof(sales_sorted, prices_sorted, left_on ='ordered_at', right_on ='updated_at', by = 'product_id', direction="nearest")

In [25]:
#compare "ordered_at" and nearest "updated_at" timestamps to figure out correct updated listed price for each order
data['listed_price'] = np.where(data['ordered_at'] >= data['updated_at'],
                                         data['new_price'], data['old_price'])
print(data.head(10))

   product_id          ordered_at  quantity_ordered  old_price  new_price  \
0     3954203 2018-09-11 01:43:00                 1      68800      60000   
1     4085861 2018-09-11 06:26:00                 1      60000      53500   
2     4085861 2018-09-11 06:53:00                 1      60000      53500   
3     4085861 2018-09-11 08:24:00                 1      60000      53500   
4     4085861 2018-09-11 09:30:00                 1      60000      53500   
5     4085861 2018-09-11 11:06:00                 1      60000      53500   
6     3954203 2018-09-11 11:11:00                 1      60000      64000   
7     3954203 2018-09-11 11:11:00                 1      60000      64000   
8     4085861 2018-09-11 11:34:00                 1      60000      53500   
9     4085861 2018-09-11 11:47:00                 2      60000      53500   

           updated_at  listed_price  
0 2018-09-10 16:32:00         60000  
1 2018-09-11 08:51:00         60000  
2 2018-09-11 08:51:00         60000  


In [26]:
#calculate revenue
summary_data = data[["product_id","ordered_at","quantity_ordered","listed_price"]].sort_values(by =["product_id","ordered_at"])
summary_data["total_revenue"] = summary_data["quantity_ordered"] * summary_data["listed_price"] 
print(summary_data)

     product_id          ordered_at  quantity_ordered  listed_price  \
26           64 2018-09-11 19:32:00                 1        239000   
39           64 2018-09-12 13:23:00                 1        239000   
42           64 2018-09-12 16:00:00                 1        239000   
79           64 2018-09-14 20:10:00                 1        239000   
0       3954203 2018-09-11 01:43:00                 1         60000   
..          ...                 ...               ...           ...   
162     4085861 2018-09-18 15:56:00                 1         52000   
168     4085861 2018-09-18 18:52:00                 1         52000   
170     4085861 2018-09-18 20:23:00                 1         52000   
171     4085861 2018-09-18 20:43:00                 1         52000   
172     4085861 2018-09-18 20:54:00                 1         52000   

     total_revenue  
26          239000  
39          239000  
42          239000  
79          239000  
0            60000  
..             ...  


In [30]:
#group total revenue by product id & listed price level
result = summary_data.groupby(by =["product_id","listed_price"]).agg({"total_revenue": np.sum})
print(result)

                         total_revenue
product_id listed_price               
64         239000               956000
3954203    57500                 57500
           60000                180000
           64000                640000
3998909    15500                 15500
           16500                231000
           17000                 34000
4085861    52000               1040000
           53500               2140000
           58000               2204000
           60000                180000
           62500               1812500
           67000                871000
